# Experimento 1
***
- Conjuntos de dados: Montgomery
- Testando a equalização de histograma em comparação com a mudança típica de escala

### Importando os pacotes necessários

In [1]:
import glob
import re
from tqdm import tqdm
import urllib.request
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

### Pré-processando os dados

In [2]:
# coletando o caminho dos arquivos dos dados do hospital montgomery
filelist_montgomery = glob.glob('../datasets/archive/Montgomery/MontgomerySet/CXR_png/*.png')

In [3]:
# quantidade de imagens disponíveis no dataset
print('quantidade de imagens:', str(len(filelist_montgomery)))

quantidade de imagens: 138


In [4]:
def extract_label(file_list):
    
    # inicializando uma lista vazia
    labels = []
    
    # iterando na lista de arquivos
    for file in tqdm(file_list):
        # detectando as classes presentes no nome da imagem
        current_label = re.findall('[0-9]{4}_(.+?).png', file)
        # adicionando a lista de rótulos as classes correspondentes a cada uma das imagens
        labels.append(current_label[0])
        
    return labels

In [5]:
# extraindo os rótulos
labels = extract_label(filelist_montgomery)

100%|██████████████████████████████████████████████████████████████████████████████| 138/138 [00:00<00:00, 8841.98it/s]


In [6]:
# visualizando a quantidade de rótulos
print('quantidade de rótulos:', str(len(labels)))

quantidade de rótulos: 138


In [7]:
# criando um dataframe com os caminhos das imagens
full_data = pd.DataFrame(filelist_montgomery, columns = ['filepath'])
# adicionando os rótulos em cada imagem
full_data['target'] = labels

In [8]:
# modificando o formato dos dados para float32
dict_type = {'target': 'float32'}
full_data = full_data.astype(dict_type)

In [9]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [10]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [11]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['filepath']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['filepath']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['filepath']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 88
quantidade de rótulos de treinamento: 88
quantidade de imagens de teste: 28
quantidade de rótulos de teste: 28
quantidade de imagens de validação: 22
quantidade de rótulos de validação: 22


### Aplicando mudança de escala típica

In [12]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255., rotation_range = 20, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'filepath',
                                                  y_col = 'target',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'raw',
                                                  target_size = (256, 256))

Found 88 validated image filenames.
Found 22 validated image filenames.
Found 28 validated image filenames.


### Preparando a rede neural convolucional

In [13]:
# callback para salvar os pesos que tiveram o melhor desempenho nos dados de validação
filepath="transferlearning_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [14]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [15]:
# transferência de aprendizado
conv_base = VGG19(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

In [16]:
# visualizando a arquitetura utilizada
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [18]:
# conectando a arquitetura vgg19 a uma rede neural densa
model = models.Sequential()
model.add(conv_base)
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(units = 128, activation = 'relu'))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Dense(units = 1, activation = 'sigmoid'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 24,220,993
Trainable params: 24,219,969
Non-trainable params: 1,024
______________________________________

In [20]:
# compilando o modelo
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [21]:
# treinando a rede 
history = model.fit_generator(train_generator, steps_per_epoch = 88 // 32, 
                              validation_data = valid_generator, callbacks = callbacks, epochs = 100)

Epoch 1/100
1/2 [==============>...............] - ETA: 7:15 - loss: 0.7563 - acc: 0.6562

KeyboardInterrupt: 